### Working with Python and Sql Countdown

In [1]:
# We will: 
        #1. Read records from an Excel sheet
        #2. Insert records to a SQLite database
        #3. run ad-hoc queries against the database records
        #4. visualize trends and patterns in the records

## 1. Read records from an excel sheet

In [4]:
import pandas as pd
import sqlite3

In [5]:
# read the public debt file
data_source = "C:\\Users\\Lenovo\\OneDrive\\Documents\\UADSL\\PYTHON\\Public Debt (Ksh Million).csv"
df = pd.read_csv(data_source)

In [6]:
df

,Year,Month,Domestic Debt,External Debt,Total
0,2021,December,"4,032,368.18","4,174,371.53","8,206,739.71"
1,2021,November,"4,008,077.30","4,109,282.85","8,117,360.15"
2,2021,October,"3,959,523.30","4,083,124.54","8,042,647.84"
3,2021,September,"3,937,777.45","4,062,504.78","8,000,282.23"
4,2021,August,"3,864,875.78","4,054,862.85","7,919,738.63"
...,...,...,...,...,...
257,2000,July,"202,362.00","394,667.37","597,029.37"
258,2000,June,"206,127.00","395,694.00","601,821.00"
259,2000,March,"201,463.22","396,800.00","598,263.22"
260,1999,December,"190,300.00","311,953.00","502,253.00"


In [5]:
df.shape

(262, 5)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           262 non-null    int64 
 1   Month          262 non-null    object
 2   Domestic Debt  262 non-null    object
 3   External Debt  262 non-null    object
 4   Total          262 non-null    object
dtypes: int64(1), object(4)
memory usage: 10.4+ KB


In [7]:
df["Domestic Debt"] 

0      4,032,368.18
1      4,008,077.30
2      3,959,523.30
3      3,937,777.45
4      3,864,875.78
           ...     
257      202,362.00
258      206,127.00
259      201,463.22
260      190,300.00
261      183,417.00
Name: Domestic Debt, Length: 262, dtype: object

In [8]:
df

,Year,Month,Domestic Debt,External Debt,Total
0,2021,December,"4,032,368.18","4,174,371.53","8,206,739.71"
1,2021,November,"4,008,077.30","4,109,282.85","8,117,360.15"
2,2021,October,"3,959,523.30","4,083,124.54","8,042,647.84"
3,2021,September,"3,937,777.45","4,062,504.78","8,000,282.23"
4,2021,August,"3,864,875.78","4,054,862.85","7,919,738.63"
...,...,...,...,...,...
257,2000,July,"202,362.00","394,667.37","597,029.37"
258,2000,June,"206,127.00","395,694.00","601,821.00"
259,2000,March,"201,463.22","396,800.00","598,263.22"
260,1999,December,"190,300.00","311,953.00","502,253.00"


In [9]:
# Domestic Debt
df['Domestic Debt'] = df['Domestic Debt'].apply(lambda x : x.replace(',',''))
df['Domestic Debt'] = df['Domestic Debt'].apply(lambda x : float(x))

In [10]:
# External debt
df['External Debt'] = df['External Debt'].apply(lambda x : x.replace(',',''))
df['External Debt'] = df['External Debt'].apply(lambda x : float(x))

# Total
df['Total'] = df['Total'].apply(lambda x : x.replace(',',''))
df['Total'] = df['Total'].apply(lambda x : float(x))

In [11]:
df[-5:]

,Year,Month,Domestic Debt,External Debt,Total
257,2000,July,202362.00,394667.37,597029.37
258,2000,June,206127.00,395694.00,601821.00
259,2000,March,201463.22,396800.00,598263.22
260,1999,December,190300.00,311953.00,502253.00
261,1999,September,183417.00,320137.00,503554.00


#### CREATE TABLE TO HOLD RECORDS FROM CSV/EXCEL

In [12]:
#### ⁠⁠2. Create table to hold records from csv/excel
# This begins by building a schema for the tables. Column names and column types must match/align to avoid errors

In [13]:
# build the schema OR take care of column naming and data typing.
tbl_columns = list(df.columns)
tbl_columns = ["Year","Month","Domestic_Debt","External_Debt","Total"]
tbl_column_types = ['integer','varchar','float','float','float']

In [14]:
tbl_schema = dict(zip(tbl_columns,tbl_column_types))
tbl_schema

{'Year': 'integer',
 'Month': 'varchar',
 'Domestic_Debt': 'float',
 'External_Debt': 'float',
 'Total': 'float'}

## **METHOD 1:**

In [15]:
# Here, we write all the columns by hand. This can be very tedious and time consuming when we have very large/wide data format - with hundreds of columns.

In [16]:
# connect to the hr database
cnx = sqlite3.connect("C:\\Users\\Lenovo\\Downloads\\hr")
cursor = cnx.cursor()

# drop the public_debt_tbl if exists
drop_tbl_sql = "DROP TABLE IF EXISTS public_debt_tbl;"
cursor.execute(drop_tbl_sql)

# create the public_debt_tbl if it doesn't exist
create_tbl_sql = '''
                    CREATE TABLE IF NOT EXISTS public_debt_tbl(
                         "Year" integer,
                         "Month" varchar(15),
                         "Domestic Debt" float,
                         "External Debt" float,
                         "Total" float
                    )
                 '''

cursor.execute(create_tbl_sql)

**METHOD 2:**

In [17]:
# Here, we parse the column names dymanically and autonomously. This is the preferred approach because it 
# saves time and handles wide/long format data.

In [18]:
# dynamic construction of the table columns/headers
sql_root = "CREATE TABLE IF NOT EXISTS public_debt_tbl("
schema_sql = ""

for key,val in tbl_schema.items():
    if key == 'Total':
        schema_sql = schema_sql + f"{key} {val}"
    else:
        schema_sql = schema_sql + f"{key} {val}, "

public_debt_tbl_sql = sql_root + schema_sql + ')'
public_debt_tbl_sql

'CREATE TABLE IF NOT EXISTS public_debt_tbl(Year integer, Month varchar, Domestic_Debt float, External_Debt float, Total float)'

In [19]:
public_debt_tbl_sql

'CREATE TABLE IF NOT EXISTS public_debt_tbl(Year integer, Month varchar, Domestic_Debt float, External_Debt float, Total float)'

In [20]:
# connect to the hr database
cnx = sqlite3.connect("C:\\Users\\Lenovo\\Downloads\\hr")
cursor = cnx.cursor()

# drop the public_debt_tbl if exists
drop_tbl_sql = "DROP TABLE IF EXISTS public_debt_tbl;"
cursor.execute(drop_tbl_sql)

# create the public_debt_tbl if it doesn't exist
cursor.execute(public_debt_tbl_sql)

**View the columns that were created on this table**

In [21]:
columns_sql = "pragma table_info('public_debt_tbl')"
table_columns = cursor.execute(columns_sql)
for item in table_columns.fetchall():
    print(item)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Month', 'varchar', 0, None, 0)
(2, 'Domestic_Debt', 'float', 0, None, 0)
(3, 'External_Debt', 'float', 0, None, 0)
(4, 'Total', 'float', 0, None, 0)


#### ⁠⁠3. Insert records from the csv/excel sheet into the sqlite table

In [22]:
#### ⁠⁠3. Insert records from the csv/excel sheet into the sqlite table

''' Use the if_exists parameter to set behavior if the table already exists.
  **fail**: Raise a ValueError.*
    **replace: Drop the table before inserting new values.
    **append: Insert new values to the existing table.
'''

' Use the if_exists parameter to set behavior if the table already exists.\n  **fail**: Raise a ValueError.*\n    **replace: Drop the table before inserting new values.\n    **append: Insert new values to the existing table.\n'

In [23]:
df.to_sql('public_debt_tbl',cnx,if_exists='replace', index=False)

262

In [24]:
# View Inserted Records

In [25]:
sql = '''
        SELECT * FROM public_debt_tbl
     '''
tbl_df = pd.read_sql(sql, cnx)
tbl_df

,Year,Month,Domestic Debt,External Debt,Total
0,2021,December,4032368.18,4174371.53,8206739.71
1,2021,November,4008077.30,4109282.85,8117360.15
2,2021,October,3959523.30,4083124.54,8042647.84
3,2021,September,3937777.45,4062504.78,8000282.23
4,2021,August,3864875.78,4054862.85,7919738.63
...,...,...,...,...,...
257,2000,July,202362.00,394667.37,597029.37
258,2000,June,206127.00,395694.00,601821.00
259,2000,March,201463.22,396800.00,598263.22
260,1999,December,190300.00,311953.00,502253.00


#### 4. Run ad-hoc queries against the database record.

In [26]:
# *Write a query to show the domestic debt totals for the last 5 years*
sql = '''
        SELECT * FROM public_debt_tbl
        WHERE Year >= 2021 - 4
     '''
tbl_df = pd.read_sql(sql, cnx)
dd_df = tbl_df[["Year","Month","Domestic Debt"]]
dd_df

,Year,Month,Domestic Debt
0,2021,December,4032368.18
1,2021,November,4008077.30
2,2021,October,3959523.30
3,2021,September,3937777.45
4,2021,August,3864875.78
5,2021,July,3792146.89
6,2021,June,3697093.18
7,2021,May,3686891.72
8,2021,April,3632914.80
9,2021,March,3569840.57


#### 5. Run ad-hoc queries against the database record.

In [27]:
## *Show the domestic debt trends for the last 5 years from the table records*

In [28]:
sql = '''
        SELECT * FROM public_debt_tbl
        WHERE Year >= 2021 - 4
     '''
tbl_df = pd.read_sql(sql, cnx)
dd_df = tbl_df[["Year","Month","Domestic Debt"]]
dd_df.head(20)

,Year,Month,Domestic Debt
0,2021,December,4032368.18
1,2021,November,4008077.30
2,2021,October,3959523.30
3,2021,September,3937777.45
4,2021,August,3864875.78
5,2021,July,3792146.89
6,2021,June,3697093.18
7,2021,May,3686891.72
8,2021,April,3632914.80
9,2021,March,3569840.57
